In [ ]:
import sys
sys.path.append('..')
from tool import *
def get_stat(sw_mode,sza_L_mode,band_mode):
    _,sza_L=torch.load(root_path+'sza_and_szaL.pth')
    if band_mode=='shortwave':  
        WB_sky_fraction=torch.load("/data/hk/albedo/white_sky_fraction/white_sky_fraction2.pth")
    else:
        if band_mode not in ['nir','vis']:
            raise('Error band_mode!')
        WB_sky_fraction=torch.load(f"/data/hk/albedo/white_sky_fraction/white_sky_fraction_{band_mode}2.pth")
    landtypes=['landtype'+str(i) for i in range(1,18)]+['snow']
    stat={}
    for month in range(1,13):
        NC=nc.Dataset(root_path+f'information_fill2/month_{month}.nc','r')
        for year in range(2001,2021):
            types_stat_albedo=[]
            types_stat_area=[]
            types_stat_energy=[]
            if sw_mode=='mean':
                sw=torch.load(f"/data/hk/albedo/kernel/{month}_SW_1degree_kernel2.pth")###
            else:
                raise('Error sw_mode')
            for type in landtypes:
                now=datetime.datetime.strptime(f'2001-{month}-1', '%Y-%m-%d')
                month_time=(now+relativedelta(months=1)-now).total_seconds()
                bs=NC.variables[f'albedo_BSA_{band_mode}-{type}'][year-2001]
                ws=NC.variables[f'albedo_WSA_{band_mode}-{type}'][year-2001]
                area_type=NC.variables[f'area-{type}'][year-2001] 
                area_land=NC.variables[f'area-land'][year-2001] 
                wf=WB_sky_fraction[(year-2001)*12+month-1]
                if sza_L_mode=='70':
                    area_type[sza_L[year-2001,month-1]>=1]=np.nan###
                elif sza_L_mode=='85':
                    area_type[sza_L[year-2001,month-1]>=2]=np.nan###
                else:
                    raise('Error sza_L_mode')
                # area=NC.variables['area-grid'][year-2001]
                if band_mode=='shortwave': 
                    types_stat_energy.append((1-(ws*wf+bs*(1-wf))/1000)*area_type*1e6*sw)
                types_stat_albedo.append(((ws*wf+bs*(1-wf))/1000))
                
                types_stat_area.append(area_type)
                                #   (1-(ws*wf+bs*(1-wf))/1000)*area_type*1e6*month_time
                # types_stat.append((ws*wf+bs*(1-wf))*area_type)
            # types_stat=np.stack(types_stat)
            if band_mode=='shortwave': 
                types_stat_energy=np.stack(types_stat_energy)
            types_stat_albedo=np.stack(types_stat_albedo)
            types_stat_area=np.stack(types_stat_area)
            # stat[f'{year}-{month}']=types_stat  
            
            types_stat_albedo[types_stat_albedo==0]=np.nan
            if band_mode=='shortwave': 
                stat[f'{year}-{month}-energy']=types_stat_energy
            stat[f'{year}-{month}-albedo']=types_stat_albedo
            stat[f'{year}-{month}-typearea']=types_stat_area
            stat[f'{year}-landarea']=area_land   
        NC.close() 
    for year in range(2001,2021):
        year_stat_albedo_weightarea=[]
        year_stat_energy=[]
        for month in range(1,13):
            if band_mode=='shortwave': 
                year_stat_energy.append(np.nansum(stat[f'{year}-{month}-energy'],axis=0))
            year_stat_albedo_weightarea.append(np.nansum((stat[f'{year}-{month}-albedo'].data*stat[f'{year}-{month}-typearea'].data/stat[f'{year}-landarea'].data),axis=0))
        year_stat_albedo_weightarea=np.stack(year_stat_albedo_weightarea)
        year_stat_albedo_weightarea[year_stat_albedo_weightarea==0]=np.nan
        if band_mode=='shortwave': 
            year_stat_energy=np.nansum(year_stat_energy,axis=0)
            stat[f'{year}-energy']=year_stat_energy
        year_stat_albedo=np.nanmean(year_stat_albedo_weightarea,axis=0)
        stat[f'{year}-albedo']=year_stat_albedo
    save_path=f'/data/hk/albedo/stat_sw-{sw_mode}_sza-{sza_L_mode}_band-{band_mode}_kernel2.pth'
    print(save_path)
    torch.save(stat,save_path)###

In [ ]:
for sw_mode in ['mean']:
    for sza_L_mode in ['70','85']:
        for band_mode in ['shortwave']:
            get_stat(sw_mode,sza_L_mode,band_mode)